# JoDa kurssin Tampereen BNB projekti. Regressiomallin toteuttaminen hinnalle käyttäen NAS.
Koska en tiedä, pitääkö projekti olla yhdessä Jupyter skriptissä vai ei. Niin teen tähän aika pitkän skriptin joka tekee kaiken. Lisäksi laitan mukaan erilliset Jupyter skriptit joissa hieman paremmin ositeltu eri asiat. 
Projekti löytyy myös 
[JoDa kurssin repo](https://github.com/THalfar/JoDa) 

Tämä on hieman hätiköidysti yhteen tehty, koska sähläämiseni takia oletin dediksen olevan vasta ensi viikolla. Aikaa tähän kyllä käytin huhtikuun alussa paljon, kunnes Kaggle kisa vei mennessään. 

Alla on erikseen tähän projektiin liittyvät skriptit: 

- "tampereBNB_datan_kasittely.ipynb"  Datan esikäsittely
 
- "tampere_reg.ipynb" Itse regressiomalli jossa käytetään NAS ja pyritään eri ositteluiden tuottamien neuroverkkojen toiseksi viimeisen kerroksen ulostuloa käyttää ominaisuuksina XGBoost 

- "baseline.ipynb" Tämä on pohjamalli, joka käyttää XGBoost jotta voin verrata tuloksia siihen. Vaikutti siltä, että käytössä oleva aineisto on liian pieni syväoppimiseen, koska tämä tuotti parhaita tuloksia.



In [ ]:
# Tarvittavat kirjastot 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os 
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler 
import optuna
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from keras import regularizers, layers, optimizers, initializers
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN, ModelCheckpoint
import time 
from optuna.integration import TFKerasPruningCallback
import pickle 
from datetime import timedelta
import gc
import xgboost as xgb
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
import warnings
from tensorflow.keras.models import Model
from sklearn.model_selection import KFold


In [ ]:
# !!! Jos liian ei liian isot tulosteet rasita, kannattaa ottaa nämä mukaan. Itse pidän koko tulosteen katsomisesta, mutta se ei ehkä sovi kaikille.
# pd.set_option('display.max_row', None) 
# pd.set_option('display.max_columns', None) 

## Datan esikäsittely. 

Tässä osiossa käsittelen dataa koneoppimisen mallia varten. 

Käytännössä tämä on "tampereBNB_datan_kasittely.ipynb" johon lisätty kommentteja mitä tapahtuu. Osiossa tarkastellaan käytössä olevaa dataa ja siivotaan sitä jatkokäyttöä varten. 

In [ ]:
# Luetaan data 
df_train = pd.read_csv('./data/Tampere_BNB_training_listing.csv')
df_test = pd.read_csv('./data/Tampere_BNB_testing_listing.csv')

In [ ]:
df_train.describe()

In [ ]:
df_train.head()
df_train['Talot.'].value_counts()

In [ ]:
puuttuvat_arvot = df_train.isnull().sum()
puuttuvat_arvot = puuttuvat_arvot[puuttuvat_arvot > 0]
print(f"Puuttuvat arvot:\n {puuttuvat_arvot}")

In [ ]:
puuttuvat_arvot_prosentteina = (puuttuvat_arvot / len(df_train)) * 100
print(f"Puuttuvat arvot prosentteina:\n {puuttuvat_arvot_prosentteina}")

In [ ]:
puuttuvat_arvot_prosentteina = puuttuvat_arvot_prosentteina.sort_values() 

sns.set_theme(style="whitegrid")
sns.barplot(x=puuttuvat_arvot_prosentteina, y=puuttuvat_arvot_prosentteina.index)
plt.title('Puuttuvien arvojen prosenttiosuus sarakkeittain')
plt.xlabel('Prosenttiosuus (%)')
plt.ylabel('Sarakkeet')
plt.show()

In [ ]:
# Muutetaan Rv sarakkeen tyyppi kokonaisluvuksi
df_train['Rv'] = df_train['Rv'].astype(int)

In [ ]:
df_train['Rv'].head()

Korvataan "Kunto" sarakkeen tyhjät arvot "Ei tietoa"

In [ ]:
df_train['Kunto'] = df_train['Kunto'].fillna('Ei tietoa')
df_train['Kunto'].value_counts().plot(kind='bar')

In [ ]:
kunto_mappaus = {
    'Ei tietoa': 0,
    'huono': 1,
    'tyyd.': 2,
    'hyvä': 3,
}
df_train['Kunto'] = df_train['Kunto'].map(kunto_mappaus)
df_train.head()

In [ ]:
df_train['Kunto'].value_counts()   

In [ ]:
df_train['Hissi'].value_counts()

In [ ]:
df_train['Hissi'] = df_train['Hissi'].astype('category')
df_train['Hissi'] = df_train['Hissi'].cat.codes
df_train['Hissi'].value_counts()

In [ ]:
df_train['Kaupunginosa'].value_counts()

In [ ]:
df_train['Kaupunginosa'] = df_train['Kaupunginosa'].astype('category')   
df_train['Kaupunginosa'].value_counts().plot(kind='bar', figsize=(25,5)) 

Koska kaupunginosia on niin paljon, tein ratkaisun missä liian pienet paikat yhdistetään yhdeksi. Tämä todennäköisesti jälkikäteen mietittynä vähän huono ratkaisu, mutta mennään tällä. 

In [ ]:
raja_arvo = 0.01
maarat = df_train['Kaupunginosa'].value_counts(normalize=True)
pienet_ryhmat = maarat[maarat < raja_arvo].index
df_train['Kaupunginosa'] = df_train['Kaupunginosa'].replace(pienet_ryhmat, 'Muu')
df_train['Kaupunginosa'].value_counts().plot(kind='bar', figsize= (25,5))

In [ ]:
df_train['Kaupunginosa'].value_counts()

In [ ]:
df_train['Kaupunginosa'] = df_train['Kaupunginosa'].astype('category')
df_train['Kaupunginosa'] = df_train['Kaupunginosa'].cat.codes
df_train['Kaupunginosa'].value_counts()

In [ ]:
df_train['Asunnon tyyppi'].value_counts()

In [ ]:
tyyppimat = {
    'Yksiö' : 1,
    'Kaksi huonetta' : 2,
    'Kolme huonetta' : 3,
    'Neljä huonetta tai enemmän' : 4
}
df_train['Asunnon tyyppi'] = df_train['Asunnon tyyppi'].map(tyyppimat)
df_train['Asunnon tyyppi'].value_counts()

In [ ]:
# Tarkastellaan mitä arvoja tämä syönyt 
df_train['Huoneisto'].value_counts()

Alhaalla alan muokkailla "Huoneisto" sarakkeen arvoja. Siellä vaikutti olevna paljon epästandartinmukaisia. Tuli hieman harjoiteltua myös regex pitkästä aikaa. 

In [ ]:
df_train['Huoneisto'] = df_train['Huoneisto'].str.lower()
df_train['Huoneisto'] = df_train['Huoneisto'].str.replace(' ', '')
df_train['Huoneisto'] = df_train['Huoneisto'].str.replace('+', ',')
df_train['Huoneisto'] = df_train['Huoneisto'].str.replace('...', "")
df_train['Huoneisto'].value_counts()


In [ ]:
df_train['Huoneisto'] = df_train['Huoneisto'].str.replace('/', ',')
df_train['Huoneisto'] = df_train['Huoneisto'].str.replace('[0-9]+h', '', regex=True)
df_train['Huoneisto'].value_counts()

In [ ]:
df_train['Huoneisto'] = df_train['Huoneisto'].str.replace('^,', '',regex=True)
df_train['Huoneisto'] = df_train['Huoneisto'].str.replace('^[-0-9]+', '',regex=True)
df_train['Huoneisto'] = df_train['Huoneisto'].str.replace(',$', '',regex=True)
df_train['Huoneisto'] = df_train['Huoneisto'].str.replace('^,', '',regex=True)
df_train['Huoneisto'].value_counts()

In [ ]:
huoneisto_split = df_train['Huoneisto'].str.split(',')
exploded = huoneisto_split.explode()
# exploded_unique = exploded.nunique()
exploded_unique_count = exploded.value_counts()
print(f"Unique values: {exploded_unique_count}")


Termeille löytyi wikistä "yleisesti" hyväksytyt lyhenteet. Joten yritän näiden mukaisesti epämääräisiä arvoja parsia. Alla on tavoitteena saada erikseen irti eri "Huoneisto" sarakkeen termejä omiksi kategoriokseen. 

In [ ]:
# Käytetään wikisivustoa https://fi.wikipedia.org/wiki/Luettelo_asuntokaupassa_k%C3%A4ytett%C3%A4vist%C3%A4_lyhenteist%C3%A4 termeille 

import re

huoneisto_split = df_train['Huoneisto'].str.split(',')

huoneisto_split = huoneisto_split.apply(lambda lst: ["parveke" if  re.search('^p$|^parv$', item) else item for item in lst])
huoneisto_split = huoneisto_split.apply(lambda lst: ["wc" if  re.search('^w$', item) else item for item in lst])
huoneisto_split = huoneisto_split.apply(lambda lst: ["sauna" if  re.search('^s$', item) else item for item in lst])
huoneisto_split = huoneisto_split.apply(lambda lst: ["2wc" if  re.search('^erill.wc$|^2xwc$', item) else item for item in lst])
huoneisto_split = huoneisto_split.apply(lambda lst: ["kph" if  re.search('^kh$', item) else item for item in lst])
huoneisto_split = huoneisto_split.apply(lambda lst: ["alkovi" if  re.search('^alk$', item) else item for item in lst])
huoneisto_split = huoneisto_split.apply(lambda lst: ["lasit" if  re.search('^l', item) else item for item in lst])
huoneisto_split = huoneisto_split.apply(lambda lst: ["avok" if  re.search('^avokeitti&#$', item) else item for item in lst])


exploded = huoneisto_split.explode()
exploded_lkm = exploded.value_counts()
print(f"Unique values: {exploded_lkm}")

Koska jotain kummallisuuksia löytyi, joita oli vain pieni määrä. Korvataan nämä "määrittämätön" arvolla

In [ ]:
minimi_lkm = 10
suodatetut_idx = exploded_lkm[exploded_lkm >= minimi_lkm].index
suodatettu_lista = huoneisto_split.apply(lambda lst: [item for item in lst if item in suodatetut_idx])
suodatettu_lista = suodatettu_lista.apply(lambda lst: ["määrittämätön"] if len(lst) == 0 else lst)

df_train['Huoneisto'] = suodatettu_lista.apply(lambda lst: ','.join(lst))

# Tarkistetaan vielä vähän tymästi, että onko kaikki kunnossa
huoneisto_split = df_train['Huoneisto'].str.split(',')
exploded = huoneisto_split.explode()
exploded_lkm = exploded.value_counts()
print(f"Uniikit arvot values: {exploded_lkm}")
print(f"Määrä arvot: {exploded_lkm.count()}")


In [ ]:
df_train['Huoneisto'].head()

In [ ]:
print(df_train.shape)
# Splitataan 'Huoneisto' -sarake ja muunnetaan se listaksi pilkun perusteella
split_data = df_train['Huoneisto'].str.split(',')

# Käytetään explode()-metodia muuntaaksemme listan elementit omiksi riveikseen
exploded_data = split_data.explode()

# Valinnainen: Suodatetaan pois harvinaiset kategoriat ennen one-hot-enkoodausta
# value_counts = exploded_data.value_counts()
# to_keep = value_counts[value_counts >= 5].index
# filtered_data = exploded_data[exploded_data.isin(to_keep)]

# Suoritetaan one-hot-enkoodaus
one_hot_encoded = pd.get_dummies(exploded_data)

# Summataan yhteen samat rivit, koska explode() luo useita rivejä samalle alkuperäiselle indeksille
one_hot_summed = one_hot_encoded.groupby(one_hot_encoded.index).sum()

# Yhdistetään one-hot-enkoodatut sarakkeet takaisin alkuperäiseen DataFrameen
df_train = df_train.join(one_hot_summed)

print(df_train.shape)

print(df_train.head())


In [ ]:
df_train["Talot."].value_counts()
df_train["Talot."] = df_train["Talot."].astype('category')
df_train["Talot."] = df_train["Talot."].cat.codes
df_train["Talot."].value_counts()

In [ ]:
df_train["Krs"].value_counts()

Päätin korvata puuttuvat arvot 0/0 kerroksella. 

In [ ]:
df_train["Krs"].isnull().sum()
df_train["Krs"] = df_train["Krs"].fillna("0/0")

In [ ]:
df_train["Krs"] = df_train["Krs"].str.replace('^-', '', regex=True)
df_train["Krs"].head()

In [ ]:
kerros_split = df_train["Krs"].str.split('/', expand=True)  
df_train["kerros"] = kerros_split[0].astype(int)
df_train["max_kerros"] = kerros_split[1].astype(int)    

In [ ]:
df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_train_filtered = df_train.drop(['Huoneisto', 'Krs'], axis=1)
df_train_filtered.info()

In [ ]:
# Kommentoin tämä pois, mutta päätin pitää jatkuvuuden kannalta. Koska tämä yksi skripti vedetään putkeen epäjärjevästi. Olen epävarma, joten parempi ratkaisu että yksi typerä putkiskripti ja sitten eriteltynä miten alunperin tein.
# df_train_filtered.to_pickle('./data/df_train_filtered.pkl')

## Regressiomalli käyttäen NAS kaikilla opetus/validointi ositteluille ja lopussa XGBoost yhdistämässä näitä 

In [ ]:
# Koska Talot. tyyppejä ei ole kovin montaa, niin yhdistetään ne kaupunginosan kanssa jonka mukaan tehdään testi data setti

df_train_filtered['combined'] = df_train_filtered[['Kaupunginosa', 'Talot.']].astype(str).agg('-'.join, axis=1)
counts = df_train_filtered['combined'].value_counts()
df_train_filtered['combined'] = df_train_filtered['combined'].map(lambda x: 'other' if counts[x] < 2 else x)
X = df_train_filtered.drop('Hinta', axis=1)
y = df_train_filtered['Hinta']

# Splitataan data käyttäen yllä olevaa osittelujoukkona. Nyt kun tätä pohdin monen viikon tauon jälkeen, olisi parempi tapa minkä opin Kagglesta. Aikataulusähläyksen takia en nyt uskalla alkaa kokeilee sitä. 

X_train, X_test, _ , _ = train_test_split(X, y, test_size=0.1, stratify=df_train_filtered['combined'], random_state=42)
X_train.drop('combined', axis=1, inplace=True)
X_test.drop('combined', axis=1, inplace=True)



Jos tätä kukaan jaksaa edes lukea ja kiinnostaa, niin aika näppärä osittelutapa:
[Kaggle notebook](https://www.kaggle.com/code/awsaf49/planttraits2024-kerascv-starter-notebook?scriptVersionId=161494049&cellId=19)

In [ ]:
# Skaalataan numeeriset muuttujat. Koska koordinaatit ja Rv sekä m2 ei ole mielestäni sellaisia, että pitäisi murehtia train ja test setin välillä vuotaisi tietoa, niin skaalataan ne kaikki yhdessä

df_train_NN = df_train_filtered.copy()
df_train_NN[['Pituusaste', 'Leveysaste']] = MinMaxScaler().fit_transform(df_train_NN[['Pituusaste', 'Leveysaste']])
df_train_NN['Rv'] = MinMaxScaler().fit_transform(df_train_NN[['Rv']])
df_train_NN['m2'] = MinMaxScaler().fit_transform(df_train_NN[['m2']])

# One hot koodataan kategoriset muuttujat
df_hot = pd.get_dummies(df_train_NN['Kaupunginosa'], prefix='Kaupunginosa').astype('int')
df_train_NN = pd.concat([df_train_NN, df_hot], axis=1)

df_hot = pd.get_dummies(df_train_NN['kerros'], prefix='kerros').astype('int')
df_train_NN = pd.concat([df_train_NN, df_hot], axis=1)

df_hot = pd.get_dummies(df_train_NN['max_kerros'], prefix='max_kerros').astype('int')
df_train_NN = pd.concat([df_train_NN, df_hot], axis=1)

df_hot = pd.get_dummies(df_train_NN['Kunto'], prefix='Kunto').astype('int')
df_train_NN = pd.concat([df_train_NN, df_hot], axis=1)

df_hot = pd.get_dummies(df_train_NN['Hissi'], prefix='Hissi').astype('int')
df_train_NN = pd.concat([df_train_NN, df_hot], axis=1)

df_hot = pd.get_dummies(df_train_NN['Asunnon tyyppi'], prefix='Asunnon tyyppi').astype('int')
df_train_NN = pd.concat([df_train_NN, df_hot], axis=1)

df_hot = pd.get_dummies(df_train_NN["Talot."], prefix='Talot.').astype('int')
df_train_NN = pd.concat([df_train_NN, df_hot], axis=1)


df_train_NN.drop(['Kaupunginosa', 'kerros', 'max_kerros', 'Kunto', 'Hissi', 'Asunnon tyyppi', "Talot."], axis=1, inplace=True)

Tämä on hieman hatara kohta. Muistelen, että oletin tuon random_state asetuksen pakottavan traintest splitin antamaan saman osittelun. Koska tarvitsen lopussa XGBoost mallille alkuperäisen datan, että NN muokatun 

In [ ]:
X = df_train_NN.drop('Hinta', axis=1)
y = df_train_NN['Hinta']

X_train_NN, X_test_NN, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=df_train_NN['combined'], random_state=42)
df_strat = X_train_NN['combined'].reset_index(drop=True)

y_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

X_train_NN.drop('combined', axis=1, inplace=True)
X_test_NN.drop('combined', axis=1, inplace=True)

X_train_NN = X_train_NN.to_numpy().astype('float32')    
X_test_NN = X_test_NN.to_numpy().astype('float32')

y_train = y_train.to_numpy().astype('float32')
y_test = y_test.to_numpy().astype('float32')

Alla suoritetaan NAS haku käyttäen Optunaa. 

In [ ]:
# Haun nimi
study_name = '502_kiireonomamokaa'
# Montako osittelua käytettiin
folds = 5
# Montako epochia kullekin osittelulle
epochs_search = 50
# Montako satunnaista hakua kieroksella
num_random = 3
# Montako TPE hakua kieroksella
num_tpe = 3

# Aika sekuntteina jota hakuun käytetän
max_search_time = 180

# Neuroneiden maksimimäärä 
max_units_all = 

# Logaritminen virhefunktio joka parempi kuin MSE tai MAE hintaregressioon 
def rmsle_loss(y_true, y_pred):
    penalty = tf.constant(1e5, dtype=tf.float32)
    valid_mask = tf.math.greater(y_pred, 0.0)
    safe_y_pred = tf.where(valid_mask, y_pred, penalty)
    rmsle = tf.sqrt(tf.reduce_mean(tf.square(tf.math.log1p(safe_y_pred) - tf.math.log1p(y_true))))
    return tf.where(tf.reduce_any(~valid_mask), penalty, rmsle)

custom_objects = {"rmsle_loss": rmsle_loss}

# TF turhaan itkee asioita joita ei tarvitse tietää
warnings.filterwarnings("ignore", category=UserWarning)
# optuna.logging.set_verbosity(optuna.logging.WARNING)     



def create_model(trial):
        
    model = keras.Sequential()
    model.add(keras.layers.InputLayer(input_shape=(X_train_NN.shape[1],), name = 'input'))
    
    num_layers = trial.suggest_int('n_layers', 1, 3)
    max_units = max_units_all

    for i in range(num_layers):
        
        num_units = trial.suggest_int(f'n_units_{i}', 16, max_units)
        dropout_rate = trial.suggest_float(f'dropout_{i}', 0.0, 0.5)
        kernel_regularizer = regularizers.l1_l2(
            l1= trial.suggest_float(f'l1_reg_{i}', 1e-8, 1, log=True),
            l2= trial.suggest_float(f'l2_reg_{i}', 1e-8, 1, log=True)
        )
        activation = trial.suggest_categorical(f'activation_{i}', ['relu', 'elu', 'LeakyReLU', 'tanh', 'selu', 'swish'])
        
        model.add(keras.layers.Dense(num_units, activation=activation, kernel_regularizer=kernel_regularizer, name = f'layer_{i}'))

        normalize = trial.suggest_categorical(f'normalize_{i}', [True, False])
        if normalize:
            model.add(keras.layers.BatchNormalization())

        model.add(keras.layers.Dropout(rate=dropout_rate))
        
        # Valitaan näistä pienempi jolla varmistetaan ettei seuraavilla kerroksilla ala olla enemmän neuroneita
        max_units = min(max_units, num_units)  
    
    model.add(keras.layers.Dense(1, activation='linear')) 
    
    # Optimisaattorin ja oppimisnopeuden valinta
    optimizer_options = ['adam', 'rmsprop', 'Nadam', 'adamax', 'Adagrad', 'Adadelta']
    optimizer_selected = trial.suggest_categorical('optimizer', optimizer_options)
    
    if optimizer_selected == 'adam':
        optimizer = optimizers.Adam()
    elif optimizer_selected == 'rmsprop':
        optimizer = optimizers.RMSprop()
    elif optimizer_selected == 'Nadam':
        optimizer = optimizers.Nadam()
    elif optimizer_selected == 'Adagrad':
        optimizer = optimizers.Adagrad()
    elif optimizer_selected == 'Adadelta':
        optimizer = optimizers.Adadelta()
    else:
        optimizer = optimizers.Adamax()

    model.compile(optimizer=optimizer, loss=rmsle_loss, metrics=['mse', 'mae'])
    
    return model


# Edellinen toteutus oli typerä, koska se kadotti aina parhaan löydetyn mallin painot. Tässä wrapataan Optunaa varten tuo objective funktio, jotta saadaan talteen paras malli.
def get_objective(fold, study_name, study):
    
    def objective(trial):

        # Kerätään roskat. Ehkä ei näin pienissä malleissa ongelma, mutta isommissa NAS hauissa ehdoton juttu
        tf.keras.backend.clear_session()
        gc.collect()
                
        
        model = create_model(trial)

        checkpoint_filepath = f'./NN_search/{study_name}_fold{fold}_checkpoint.h5'

        callbacks = [
            TFKerasPruningCallback(trial, 'val_loss'),
            ReduceLROnPlateau('val_loss', patience=5, factor=0.5, verbose = 0), 
            TerminateOnNaN(),
            ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_loss', save_best_only=True, verbose=0)
        ]

        batch_size = trial.suggest_int('batch_size', 16, 128, log=True)    

        history = model.fit(X_train_b, y_train_b, epochs=epochs_search, validation_data=(X_val_b, y_val_b), batch_size=batch_size, verbose=0, callbacks=callbacks)
        val_loss = np.min(history.history['val_loss'])

        # Jos löydetty malli on parempi kuin paras tähän mennessä, niin tallennetaan se
        
        if trial.number > 0: # Vähän purkkaratkaisu, koska ensimmäisellä trialilla ei ole vielä best_valuea
            if val_loss < study.best_value:
                model = keras.models.load_model(checkpoint_filepath, custom_objects=custom_objects)
                savepath = f'./NN_search/{study_name}_best.h5'
                
                print('#' * 50)
                print(f'New best model found for fold {fold} with value {val_loss}')
                print(f'Saving model to {savepath}')
                print('#' * 50)
                model.save(savepath)
        else:
            print(f'First trial for fold {fold} completed with value {val_loss}')
            model = keras.models.load_model(checkpoint_filepath, custom_objects=custom_objects)
            savepath = f'./NN_search/{study_name}_best.h5'
            model.save(savepath)

        return val_loss

    return objective


####

total_time_start = time.time()  
search_time_start = time.time() 
num_completed_trials = 0
search_rounds = 0
time_taken = 0

while time_taken < max_search_time:
        
    fold = 0 
    time_fold_start = time.time()    
    skf =  StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)

    stratified_labels = df_strat
    
    for train_index, val_index in skf.split(X_train_NN, stratified_labels):

        print('-------------------')
        print(f"Starting fold {fold} search...")
        X_train_b, X_val_b = X_train_NN[train_index], X_train_NN[val_index]    
        y_train_b, y_val_b = y_train[train_index], y_train[val_index]

        fold_name = f'{study_name}_{fold}'
       
        # Jokaiselle foldille on oma study
        study = optuna.create_study(direction='minimize',
                                    pruner=optuna.pruners.MedianPruner(),
                                    study_name=fold_name,
                                    storage=f'sqlite:///tre_reg.db',
                                    load_if_exists=True                                 
                                    )
        
    
        fold_time = time.time()    

        fold_random = time.time()
        # optuna.logging.set_verbosity(optuna.logging.WARNING)     # TODO OTA POIS KUHA TOIMII
        QMCs = optuna.samplers.QMCSampler(warn_independent_sampling = False)
        TPEs = optuna.samplers.TPESampler(n_startup_trials=0, multivariate=True, warn_independent_sampling = False)

        if num_random > 0:   
            study.sampler = QMCs
            print(f'Random search for fold {fold} with {num_random} trials...')
            objective_function = get_objective(fold, fold_name, study)
            study.optimize(objective_function, n_trials=num_random)
            print(f'Time taken for random search: {str(timedelta(seconds=(time.time() - fold_random)))}')

        fold_tpe = time.time()  
        if num_tpe > 0:
            study.sampler = TPEs
            print(f'TPE search for fold {fold} with {num_tpe} trials...')
            objective_function = get_objective(fold, fold_name, study)
            study.optimize(objective_function, n_trials=num_tpe)
            print(f'Time taken for TPE search: {str(timedelta(seconds=(time.time() - fold_tpe)))}')

        num_completed_trials += num_random + num_tpe
        print('-------------------')
        print(f'Finished fold {fold} search.')
        print(f"Time taken for this fold: {str(timedelta(seconds=(time.time() - fold_time)))}")                
        print(f'Fold {fold} best value so far: {study.best_value}')
        print(f'Best parameters so far: {study.best_params}')
        print(f'Mean time for one trial this fold: {str(timedelta(seconds=(time.time() - fold_time) / (num_random + num_tpe)))}')
        print(f'This fold has made total {study.trials_dataframe().shape[0]} trials.')

        fold += 1

    search_rounds += 1
    
    time_taken = time.time() - search_time_start
    
    print(f'\n# Completed search round: {search_rounds} #')
    print(f'Time taken for all folds this round: {str(timedelta(seconds=(time.time() - time_fold_start)))}')
    print(f'Total time taken for search: {str(timedelta(seconds=(time.time() - search_time_start)))}')
    print(f'Made trials this far: {num_completed_trials}')
    print(f"Current mean time for one trial: {str(timedelta(seconds=(time.time() - search_time_start) / num_completed_trials))}\n")





Nyt meillä on kullekin foldille NAS avulla haettu viisi eri verkkoa. 
Katsotaan ensin näiden verkkojen virheet "sanitycheck" ja sitten jatketaan XGBoost pariin. 

In [ ]:
def rmsle_score(y_true, y_pred):
    return np.sqrt(mean_squared_error(np.log1p(y_true+1), np.log1p(y_pred+1)))

best_models = []

for fold in range(folds):
    
    model_savepath = f'./NN_search/{study_name}_{fold}_best.h5'
    print(f'Loading model for fold {fold} from {model_savepath}')
    model = keras.models.load_model(model_savepath, custom_objects=custom_objects)
    best_models.append(model)
    model.summary()
    predictions = model.predict(X_test_NN, verbose = 0)
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    rmsle = rmsle_score(y_test, predictions)
    
    print(f'Model for fold {fold} evaluation:')
    print(f"MSE: {mse:.3f}")
    print(f"MAE: {mae:.3f}")
    print(f"R2: {r2:.3f}")
    print(f"RMSLE: {rmsle:.3f}")
    print("-"*40)

Huhu! Koodi on kuukausi sitten tehty (29.3) ja ei palaudu hetkessä kaikki mieleen.
Hitsi, pahoittelut jos joku tätä sotkua yrittää edes lukea. 

Ideana oli Optunalla valita NAS mallit mitä napataan mukaan XGBoost. Tästä koko featureryppäästä vielä valitaan scikit-learn SelectKBest mukaisesti parhaat featuret. 
Setä kyllä ampuu niin tykillä kärpästä :D 
Muistinkin miksi Kaggle kisa vetäisi mukaansa. Baselinenä toiminut pelkkä XGBoost antoi parempia tuloksia kuin tämä härveli. Data on yksinkertaisesti liian pieni mielestäni syväoppimiseen, ettei oikein ylioppimista voi välttää. 
Olinkin unohtanut tuon ominaisuuksien valinnan joka oli mielestäni ihan fiksua. Pitääkin se napata mukaan Kaggle viritelmääni jossa ei aikataulukiirettä. 


In [ ]:
import warnings
from tensorflow.keras.models import Model
import optuna
import xgboost as xgb
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from datetime import timedelta
import time
from tensorflow.keras.models import Model
import glob
from tensorflow.keras.models import load_model
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.model_selection import train_test_split, StratifiedKFold
import warnings



# Montko quasirandom samplea ja montako tpe samplea ja kauanko tätä härveliä pyöritellään 
num_random = 3
num_tpe = 3
max_time_xgb = 180


X_train_features_list = []
X_test_features_list = []
features_names_list = []

for idx, model in enumerate(best_models):
    feature_extractor = Model(inputs=model.inputs, outputs=model.layers[-2].output)
    X_train_features = feature_extractor.predict(X_train_NN)
    X_test_features = feature_extractor.predict(X_test_NN)

    has_nan = np.isnan(X_train_features).any() or np.isnan(X_test_features).any()
    print(f'Model {idx} has nan values: {has_nan}')
    
    X_train_features_list.append(X_train_features)
    X_test_features_list.append(X_test_features)

    num_features = X_train_features.shape[1]
    model_feature_names = [f"model_{idx}_feature_{feature_idx}" for feature_idx in range(num_features)]
    features_names_list.append(model_feature_names)


original_feature_names = list(X_train.columns) 
combined_feature_names = original_feature_names + features_names_list

# Yhdistetään ominaisuusvektorit

def select_models(X_train_features_list, X_test_features_list, features_names_list, which_models):

    X_train_selected = []
    X_test_selected = []
    selected_names = []

    if all(not choosenode for choosenode in which_models):        
        X_train_selected = None
        X_test_selected = None
        selected_names = None

    else:
        for idx, choosenode in enumerate(which_models):
            if choosenode: 
                X_train_selected.append(X_train_features_list[idx])
                X_test_selected.append(X_test_features_list[idx])
                selected_names.extend(features_names_list[idx])                
            
        X_train_selected = np.concatenate(X_train_selected, axis=1)
        X_test_selected = np.concatenate(X_test_selected, axis=1)
        
        
    return X_train_selected, X_test_selected, selected_names
    

X_train_combined = np.concatenate(X_train_features_list, axis=1)
X_test_combined = np.concatenate(X_test_features_list, axis=1)

X_train_combined = np.concatenate([X_train_combined, X_train], axis=1)   
X_test_combined = np.concatenate([X_test_combined, X_test], axis=1)


def select_features(X_train_combined, X_test_combined, y_train, combined_feature_names, method, max_feature):
    
    if method == 'f_regression':
        method_function = f_regression
    elif method == 'mutual_info_regression':
        method_function = mutual_info_regression
    else:
        method_function = None

    if method_function is not None:
        selector = SelectKBest(method_function, k=max_feature)
        X_train_combined_selected = selector.fit_transform(X_train_combined, y_train)
        X_test_combined_selected = selector.transform(X_test_combined)
        selected_indices = selector.get_support(indices=True)
        selected_features_names = np.array(combined_feature_names)[selected_indices]
        selected_features_names = selected_features_names.tolist()
        
        return X_train_combined_selected, X_test_combined_selected, selected_features_names
    else:
        return X_train_combined, X_test_combined, combined_feature_names


def rmsle(y_true, y_pred):
    if np.any(y_pred <= 0):
        return 1e6
    return np.sqrt(np.mean(np.square(np.log1p(y_pred) - np.log1p(y_true))))




def objective(trial):
    # Määritetään parametrit, jotka optimoidaan XGBoosrt
    param = {
        "booster": "dart",        
        'max_depth': trial.suggest_int('max_depth', 1, 6),
        "lambda": trial.suggest_float("lambda", 1e-4, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-4, 1.0, log=True),
        'learning_rate': trial.suggest_float('learning_rate', 1e-2, 2, log=True),
        'subsample': trial.suggest_float('subsample', 0.2, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.2, 1.0),
        'gamma': trial.suggest_float('gamma', 0.1, 42, log = True),     
        'nthread' : -2
    }

    num_boost_round = trial.suggest_int('BRounds', 1, 142)  

    # Alla olevat liittyvät ominaisuuksien valitsemiseen. Valitettavasti nyt tämä härveli tukee vaan 5 foldia. Ei aikaa kiire on yöllä, että dynaamisesti tämä foldien mukaan. 
    selector = trial.suggest_categorical('S', choices = ['f_regression', 'mutual_info_regression', 'None'])
    select_0 = trial.suggest_categorical('S0', [True, False])
    select_1 = trial.suggest_categorical('S1', [True, False])
    select_2 = trial.suggest_categorical('S2', [True, False])
    select_3 = trial.suggest_categorical('S3', [True, False])
    select_4 = trial.suggest_categorical('S4', [True, False])

    X_train_combined_selected, X_test_combined_selected , selected_features_names = select_models(X_train_features_list, X_test_features_list, features_names_list, [select_0, select_1, select_2, select_3, select_4])

    if X_train_combined_selected is not None:
        combined_feature_names = selected_features_names + original_feature_names
        X_train_combined_selected = np.concatenate([X_train_combined_selected, X_train], axis=1)
        X_test_combined_selected = np.concatenate([X_test_combined_selected, X_test], axis=1)
    else:
        X_train_combined_selected = X_train
        X_test_combined_selected = X_test
        combined_feature_names = original_feature_names


    num_selected = trial.suggest_int('N_fea', 1, X_train_combined_selected.shape[1])
    X_train_combined_selected, _ , combined_feature_names = select_features(X_train_combined_selected, X_test_combined_selected, y_train, combined_feature_names, selector, num_selected)
    # print(f"Selected features: {combined_feature_names}")
    
    
    rmsle_scores = []
    dtrain_full = xgb.DMatrix(X_train_combined_selected, label=y_train, feature_names=combined_feature_names)

    skf =  StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
    stratified_labels = df_strat
    for train_index, val_index in skf.split(X_train_NN, stratified_labels):
        
        dtrain = dtrain_full.slice(train_index)
        dval = dtrain_full.slice(val_index)

        evals_result = {}
        bst = xgb.train(param, dtrain, num_boost_round=num_boost_round, evals=[(dval, 'val')], evals_result=evals_result, verbose_eval=False, early_stopping_rounds=100)
        best_iteration = bst.best_iteration
        preds = bst.predict(dval, iteration_range=(0, best_iteration + 1))
        y_true = y_train[val_index]       
        loss = rmsle(y_true, preds)
        rmsle_scores.append(loss)

    average_rmsle = np.mean(rmsle_scores)
    return average_rmsle


study = optuna.create_study(direction='minimize')

# optuna.logging.set_verbosity(optuna.logging.WARNING)
# optuna.logging.set_verbosity(optuna.logging.INFO)


time_xgb_start = time.time()

QMCsampler = optuna.samplers.QMCSampler(warn_independent_sampling = False)
TPEsampler = optuna.samplers.TPESampler(multivariate=True, n_startup_trials=0, warn_independent_sampling = False)

# Koulutuslooppi joka kouluttaa XGBoost kunnes aikaraja saavutettu
while time.time() - time_xgb_start < max_time_xgb:

    
    print(f'Random sampling {num_random} trials...')
    study.optimize(objective, n_trials=num_random)
    study.sampler = QMCsampler    
    print(f'TPE sampling {num_tpe} trials...')
    study.sampler = TPEsampler
    study.optimize(objective, n_trials=num_tpe)
    print(f'Time taken so far: {str(timedelta(seconds=(time.time() - time_xgb_start)))}')

print(f'Time taken for XGBoost optimization: {str(timedelta(seconds=(time.time() - time_xgb_start)))}')
print(f'Time taken for one trial: {str(timedelta(seconds=(time.time() - time_xgb_start) / (num_random + num_tpe)))}')

print(f"Best val: {study.best_trial.value}")
print(f'Best params: {study.best_params}')


# Nyt kun malli on valittu, niin voidaan kouluttaa se koko datalla ja testata sitä testidatalla
# Alla ensin otetaan Optunan valitsemat mallit jotka tuossa lopun listassa ovat "päälle/pois"
X_train_combined_selected, X_test_combined_selected , selected_features_names = select_models(X_train_features_list, X_test_features_list, features_names_list, [study.best_params['S0'], study.best_params['S1'], study.best_params['S2'], study.best_params['S3'], study.best_params['S4']])

if X_train_combined_selected is not None:
    combined_feature_names = selected_features_names + original_feature_names
    X_train_combined_selected = np.concatenate([X_train_combined_selected, X_train], axis=1)
    X_test_combined_selected = np.concatenate([X_test_combined_selected, X_test], axis=1)
else:
    X_train_combined_selected = X_train
    X_test_combined_selected = X_test
    combined_feature_names = original_feature_names

# Optuna myös valitsi kuinka monta parasta ominaisuutta napattiin tästä ominausuuskasasta mukaan.
X_train_combined_selected, X_test_combined_selected, selected_features_names = select_features(X_train_combined_selected, X_test_combined_selected, y_train, combined_feature_names, study.best_params['S'], study.best_params['N_fea'])

dtrain = xgb.DMatrix(X_train_combined_selected, label=y_train, feature_names=selected_features_names)
best_model = xgb.train(study.best_params, dtrain, num_boost_round=study.best_params['BRounds'])

# Testataan mallia koulutusdatalla jotta voidaan arvioda onko tässä koko hommassa mitään järkeä
pred_train = best_model.predict(dtrain)
mae_train = mean_absolute_error(y_train, pred_train)
mse_train = mean_squared_error(y_train, pred_train)
r2_train = r2_score(y_train, pred_train)
rmsle_train = rmsle(y_train, pred_train)
print(f"Train MAE: {mae_train}, Train MSE: {mse_train}, Train R2: {r2_train}, Train RMSLE: {rmsle_train}")

# Testataan mallia testidatalla  
dtest = xgb.DMatrix(X_test_combined_selected, label=y_test, feature_names=selected_features_names)
predictions = best_model.predict(dtest)

mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
rmsle_val = rmsle(y_test, predictions)
print(f"MAE: {mae}, MSE: {mse}, R2: {r2}, RMSLE: {rmsle_val}")

# Visualisoidaan ennustettuja arvoja verrattuna todellisiin arvoihin
plt.figure(figsize=(20, 10))
plt.scatter(y_test, predictions, edgecolors=(0, 0, 0))
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
plt.xlabel('Measured')
plt.ylabel('Predicted')
plt.title('Measured vs. Predicted Values')
plt.show()

# Katsellaan vielä mistä XGBoost oli kiinnostunut
fig, ax = plt.subplots(figsize=(20, 30))
xgb.plot_importance(best_model, importance_type='weight', ax=ax)
ax.set_title('Feature Importance by Weight', fontsize=16)
plt.show()

fig, ax = plt.subplots(figsize=(20, 30))
xgb.plot_importance(best_model, importance_type='gain', ax=ax)
ax.set_title('Feature Importance by Gain', fontsize=16)
plt.show()

fig, ax = plt.subplots(figsize=(20, 30))
xgb.plot_importance(best_model, importance_type='cover', ax=ax)
ax.set_title('Feature Importance by Cover', fontsize=16)
plt.show()


